# Hubbard Maximum Probability Domains

In [1]:
# Force the local gqcpy to be imported.
import sys
sys.path.insert(0, '../../build/gqcpy')

# Import the local gqcpy.
import gqcpy

# Import Python modules.
import copy
import itertools as it
import numpy as np

## Specifying the Hubbard model for benzene

We would like to investigate benzene, a 6-site cyclic chain. Since the Hubbard model works in an abstract site basis, we should define an adjacency matrix describing a chain consisting of 6 sites. A Hubbard system is described by a hopping term `t` and electrostatic repulsion term `U`. With these two parameters, we can make a hopping matrix.

In [2]:
A = gqcpy.AdjacencyMatrix.Cyclic(6) # adjacency matrix

t = 1 # hopping term
U = 5 # electrostatic repulsion

H = gqcpy.HoppingMatrix(A, t, U) # hopping matrix

array([[0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 1, 0]], dtype=uint64)

This hopping matrix is then used to construct the Hubbard model Hamiltonian.

In [4]:
hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)

In [5]:
K = 6  # number of sites
N_P = 3  # number of electron pairs

onv_basis = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)

In [6]:
solver = gqcpy.EigenproblemSolver.Dense()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

wfn = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
wfn.coefficients().shape

(400,)

In [7]:
domain_partitions = np.array(list(it.product(range(2), repeat=6)))
domain_probabilities = np.zeros(domain_partitions.shape[0])

In [9]:
def MPD(ONV_a, Ia, ONV_b, Ib):
    ONV_a, ONV_b = np.array(list(repr(ONV_a))), np.array(list(repr(ONV_b)))
    
    domain = np.where(domain_partition == 1)[0]
    complement = np.where(domain_partition == 0)[0]
    
    Na, Nb = np.sum(ONV_a[domain].astype(np.float)), np.sum(ONV_b[domain].astype(np.float))
    
    
    if Na==1 and Nb==1:
        address = onv_basis.compoundAddress(Ia, Ib)
        domain_probabilities[d] += wfn.coefficients()[address]**2
        #print(domain_partition, ONV_a, ONV_b, Na, Nb, wfn.coefficients()[address]**2)

In [10]:
for d,domain_partition in enumerate(domain_partitions):
    onv_basis.forEach(MPD)
    #print(domain_probabilities[d])

In [11]:
domain_partitions = np.delete(domain_partitions, (0), axis=0)
domain_probabilities = np.delete(domain_probabilities, (0), axis=0)
domain_partitions = np.delete(domain_partitions, (-1), axis=0)
domain_probabilities = np.delete(domain_probabilities, (-1), axis=0)

In [12]:
for dpart, dprob in zip(domain_partitions, domain_probabilities):
    print(dpart, dprob)

[0 0 0 0 0 1] 0.08381666106355992
[0 0 0 0 1 0] 0.0838166610635599
[0 0 0 0 1 1] 0.6919837996550809
[0 0 0 1 0 0] 0.08381666106355999
[0 0 0 1 0 1] 0.28004449760336697
[0 0 0 1 1 0] 0.69198379965508
[0 0 0 1 1 1] 0.08137487062580379
[0 0 1 0 0 0] 0.08381666106355992
[0 0 1 0 0 1] 0.47711153476958185
[0 0 1 0 1 0] 0.28004449760336714
[0 0 1 0 1 1] 0.09479681788045885
[0 0 1 1 0 0] 0.6919837996550807
[0 0 1 1 0 1] 0.09479681788045875
[0 0 1 1 1 0] 0.08137487062580383
[0 0 1 1 1 1] 0.0019134778052424602
[0 1 0 0 0 0] 0.08381666106356
[0 1 0 0 0 1] 0.28004449760336697
[0 1 0 0 1 0] 0.4771115347695815
[0 1 0 0 1 1] 0.0947968178804588
[0 1 0 1 0 0] 0.28004449760336725
[0 1 0 1 0 1] 0.06430286788230218
[0 1 0 1 1 0] 0.0947968178804587
[0 1 0 1 1 1] 0.006288076485571898
[0 1 1 0 0 0] 0.6919837996550803
[0 1 1 0 0 1] 0.09479681788045882
[0 1 1 0 1 0] 0.0947968178804588
[0 1 1 0 1 1] 0.006522245867108786
[0 1 1 1 0 0] 0.08137487062580367
[0 1 1 1 0 1] 0.006288076485571883
[0 1 1 1 1 0] 0.0019134

In [13]:
def single_site_flips(domain_partition):
    single_site_flip_domains = []
    
    domain_indices = np.where(domain_partition == 1)[0]
    
    for domain_index in domain_indices:
        new_domain_partition = copy.deepcopy(domain_partition)
        new_domain_partition[domain_index] = 0
        single_site_flip_domains.append(new_domain_partition)
        
    return np.array(single_site_flip_domains)
    


In [19]:
def is_stabile(domain_partition, domain_probability):
    
    domain_flips = single_site_flips(domain_partition)

    for domain_flip in domain_flips:
        domain_flip_index = np.where(np.all(domain_flip == domain_partitions, axis=1))[0]
        if domain_probability < domain_probabilities[domain_flip_index]:
            return False
    if np.where(np.all(domain_flip == domain_partitions, axis=1))[0].size > 0:
        return True
    else: 
        return False

In [21]:
for domain_partition, domain_probability in zip(domain_partitions, domain_probabilities):
    if is_stabile(domain_partition, domain_probability):
        print(domain_partition, domain_probability)

[0 0 0 0 1 1] 0.6919837996550809
[0 0 0 1 0 1] 0.28004449760336697
[0 0 0 1 1 0] 0.69198379965508
[0 0 1 0 0 1] 0.47711153476958185
[0 0 1 0 1 0] 0.28004449760336714
[0 0 1 1 0 0] 0.6919837996550807
[0 1 0 0 0 1] 0.28004449760336697
[0 1 0 0 1 0] 0.4771115347695815
[0 1 0 1 0 0] 0.28004449760336725
[0 1 1 0 0 0] 0.6919837996550803
[1 0 0 0 0 1] 0.6919837996550802
[1 0 0 0 1 0] 0.2800444976033671
[1 0 0 1 0 0] 0.4771115347695816
[1 0 1 0 0 0] 0.280044497603367
[1 1 0 0 0 0] 0.6919837996550804


/Users/dariatolstykh/opt/anaconda3/envs/gqcp_dev/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  import sys
